In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 14.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import warnings
warnings.filterwarnings(action='ignore')
from catboost import *
import sklearn
import math
from sklearn.preprocessing import LabelEncoder
import math

In [3]:

def log(train,test):
    col_list = train.columns
    for col in col_list :
        if '제곱' in col:
            train[col+'log'] = np.log1p(train[col])
            test[col+'log'] = np.log1p(test[col])
    return train,test

def del_columns(train, test):
    col_list = train.columns
    nan_list = []
    nan_cnt = []
    nan_col = []
    full_list = []
    for col in col_list:
        if train[col].isnull().sum() == 0 :
            full_list.append(col)
            continue
        nan_list.append([col, train[col].isnull().sum()])
        nan_cnt.append(train[col].isnull().sum())
        nan_col.append(col)

    '''모든값이 결측값이면 제거'''
    del_col = []
    for col in nan_list :
        if col[1] == len(train) :
            del_col.append(col[0])
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)

    '''값이 1개 존재하면 제거'''
    del_col = []
    col_list = train.describe().columns
    for col in col_list :
        if col == 'Y_Class':
            continue
        if col == 'Y_Quality':
            continue
        if col == 'LINE':
            continue
        if col == 'PRODUCT_CODE':
            continue
        if train[col].nunique()==1 :
            del_col.append(col)
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    return train,test
   
def make_train_test_dataset(train,test):
    train_x = train.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
    test_x = test.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
    train_y = train['Y_Quality']
    train_w = train[['Y_Class']]
    return train_x, test_x, train_y, train_w

def fillna(train,test,value):
    train = train.fillna(value)
    test = test.fillna(value)
    return train,test

def labelencoder(train,test,col_list):
    qual_col = col_list
    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i]) 
    return train,test

def multi_col(train,test) : 
    col_list = train.columns
    for col in col_list :
        if col == 'LINE':
            continue
        train[col+'제곱'] = (train[col]**2)//100
        test[col+'제곱'] = (test[col]**2)//100
    return train,test

In [4]:
train = pd.read_csv('/content/drive/MyDrive/LGaimers/train.csv')
test = pd.read_csv('/content/drive/MyDrive/LGaimers/test.csv')

# y quality scaling
train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 1866) (67, 1866) (343, 550) (239, 550)


In [5]:
from catboost import *

In [6]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

rfe_A = RFE(estimator=CatBoostRegressor(random_state=1234,verbose=500,iterations=1500,learning_rate=0.033), n_features_to_select=None)
rfe_T = RFE(estimator=CatBoostRegressor(random_state=1234,verbose=500,iterations=1500,learning_rate=0.033), n_features_to_select=None)
rfe_O = RFE(estimator=CatBoostRegressor(random_state=1234,verbose=500,iterations=1500,learning_rate=0.033), n_features_to_select=None)

In [7]:
trainA_31_x

,LINE,X_128,X_129,X_130,X_131,X_132,X_133,X_134,X_136,X_137,...,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870
0,2,7813.0,7813.0,-1.0,-1.0,0.19,0.20,0.19,228.0,228.0,...,197.286667,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77
1,3,-1.0,-1.0,19854.0,19854.0,0.20,0.21,0.20,413.0,414.0,...,193.296552,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55
2,2,7815.0,7815.0,-1.0,-1.0,0.19,0.20,0.19,228.0,228.0,...,179.820690,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35
3,3,-1.0,-1.0,19856.0,19856.0,0.20,0.21,0.20,414.0,414.0,...,181.920690,165.8,384.0,369.188679,353.0,37.74,39.17,52.17,30.58,71.78
4,2,7817.0,7817.0,-1.0,-1.0,0.19,0.20,0.18,228.0,228.0,...,196.393333,182.6,383.0,367.351852,352.0,38.70,41.89,46.93,33.09,76.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,2,11920.0,11920.0,-1.0,-1.0,0.19,0.20,0.19,230.0,230.0,...,191.993103,181.6,394.0,371.943396,353.0,51.71,59.64,54.61,57.05,63.18
584,0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.0,-1.0,...,195.682143,184.9,466.0,448.634615,432.0,-1.00,-1.00,-1.00,-1.00,-1.00
585,1,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,-1.0,-1.0,...,187.396552,176.7,472.0,450.339623,432.0,-1.00,-1.00,-1.00,-1.00,-1.00
594,2,14810.0,14810.0,-1.0,-1.0,0.19,0.20,0.19,304.0,304.0,...,180.810345,168.7,384.0,369.811321,353.0,49.47,53.07,50.89,55.10,66.49


In [ ]:
rfe_A.fit(trainA_31_x, trainA_31_y)
rfe_T.fit(train_T_31_x, train_T_31_y)
rfe_O.fit(train_O_31_x, train_O_31_y)

0:	learn: 0.0183320	total: 158ms	remaining: 3m 56s
500:	learn: 0.0012455	total: 53.9s	remaining: 1m 47s
1000:	learn: 0.0001219	total: 1m 40s	remaining: 50s
1499:	learn: 0.0000126	total: 2m 27s	remaining: 0us
0:	learn: 0.0183528	total: 112ms	remaining: 2m 48s
500:	learn: 0.0012249	total: 45.5s	remaining: 1m 30s
1000:	learn: 0.0001156	total: 1m 33s	remaining: 46.4s
1499:	learn: 0.0000119	total: 2m 19s	remaining: 0us
0:	learn: 0.0183702	total: 109ms	remaining: 2m 42s
500:	learn: 0.0012967	total: 46s	remaining: 1m 31s
1000:	learn: 0.0001197	total: 1m 32s	remaining: 46.3s
1499:	learn: 0.0000115	total: 2m 19s	remaining: 0us
0:	learn: 0.0183366	total: 219ms	remaining: 5m 27s
500:	learn: 0.0012179	total: 46.8s	remaining: 1m 33s
1000:	learn: 0.0001149	total: 1m 32s	remaining: 46.3s
1499:	learn: 0.0000117	total: 2m 20s	remaining: 0us
0:	learn: 0.0183755	total: 113ms	remaining: 2m 49s
500:	learn: 0.0012330	total: 45.5s	remaining: 1m 30s
1000:	learn: 0.0001338	total: 1m 32s	remaining: 46.3s
1499:	

In [ ]:
for i, col in zip(range(trainA_31_x.shape[1], trainA_31_x.columns)):
    print(f"{col} selected={rfe_A.supprot_[i]} rank={rfe_A.ranking_[i]}")

In [ ]:
for i, col in zip(range(train_T_31_x.shape[1], train_T_31_x.columns)):
    print(f"{col} selected={rfe_T.supprot_[i]} rank={rfe_T.ranking_[i]}")

In [ ]:
for i, col in zip(range(train_O_31_x.shape[1], train_O_31_x.columns)):
    print(f"{col} selected={rfe_O.supprot_[i]} rank={rfe_O.ranking_[i]}")